In [1]:
pip install albumentations opencv-python


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [2]:
import albumentations as A

IMG_SIZE = (224, 224)

heavy_aug = A.Compose([
    A.Rotate(limit=25, p=1.0),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(0.3, 0.3, p=0.8),
    A.RandomResizedCrop(
        size=IMG_SIZE,
        scale=(0.85, 1.0),   # MUST be <= 1
        ratio=(0.9, 1.1),
        p=0.8
    ),
    A.GaussianBlur(blur_limit=(3, 5), p=0.3)
])

medium_aug = A.Compose([
    A.Rotate(limit=15, p=1.0),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(0.2, 0.2, p=0.6),
    A.RandomResizedCrop(
        size=IMG_SIZE,
        scale=(0.95, 1.0),
        ratio=(0.95, 1.05),
        p=0.5
    )
])

light_aug = A.Compose([
    A.Rotate(limit=10, p=1.0),
    A.HorizontalFlip(p=0.3),
    A.RandomBrightnessContrast(0.1, 0.1, p=0.3)
])


C:\Users\rk001\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
AUG_MAP = {
    "bacterial_panicle_blight": heavy_aug,
    "bacterial_leaf_streak": heavy_aug,
    "bacterial_leaf_blight": heavy_aug,

    "downy_mildew": medium_aug

    # No entry = no augmentation
}


In [4]:
import cv2
import os
from tqdm import tqdm

INPUT_DIR = r"C:\Users\rk001\Downloads\paddy-disease-classification\train_images"
OUTPUT_DIR = "train_aug"
AUG_PER_IMAGE = 3

os.makedirs(OUTPUT_DIR, exist_ok=True)

for cls in os.listdir(INPUT_DIR):
    cls_in = os.path.join(INPUT_DIR, cls)
    cls_out = os.path.join(OUTPUT_DIR, cls)
    os.makedirs(cls_out, exist_ok=True)

    aug = AUG_MAP.get(cls)   # may be None for high-count classes

    for img_name in tqdm(os.listdir(cls_in), desc=cls):
        img_path = os.path.join(cls_in, img_name)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # save original image
        cv2.imwrite(
            os.path.join(cls_out, img_name),
            cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        )

        # 🔑 FIX: skip augmentation if aug is None
        if aug is None:
            continue

        # generate augmented images
        for i in range(AUG_PER_IMAGE):
            augmented = aug(image=image)["image"]
            aug_name = img_name.replace(".jpg", f"_aug{i}.jpg")

            cv2.imwrite(
                os.path.join(cls_out, aug_name),
                cv2.cvtColor(augmented, cv2.COLOR_RGB2BGR)
            )


bacterial_leaf_blight:   0%|          | 0/479 [00:00<?, ?it/s]

tungro: 100%|██████████| 1088/1088 [00:24<00:00, 44.83it/s]
